In [2]:
import pandas as pd
import itertools

df_1 = pd.read_csv('aggregated_results.csv')
df_2 = pd.read_csv('CRISPRGeneEffect.csv')
df_3 = pd.read_csv('Model.csv')

C:\Users\josep\AppData\Local\Temp\ipykernel_24888\42531858.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import ast
genes = df_1['All genes'].tolist()
genes = [ast.literal_eval(g) for g in genes]
genes = [item for sublist in genes for item in sublist]
genes = set(genes)
genes

{'GBAP1',
 'PRODH2',
 'SUMO1P1',
 'CCL27',
 'NOTCH2NLC',
 'AGFG2',
 'TTC23',
 'TMEM167B',
 'FANK1',
 'CIDECP1',
 'H3C13',
 'TEN1',
 'ISG20L2',
 'CLIC2',
 'SERPINA5',
 'TAAR1',
 'WRN',
 'DENND5B',
 'RAI2',
 'THUMPD3-AS1',
 'CCS',
 'CNPY2',
 'DSC3',
 'EDRF1',
 'BAALC-AS1',
 'SLC45A4',
 'PAGE2B',
 'A3GALT2',
 'PDK1',
 'RPL10A',
 'FCGBP',
 'FXYD3',
 'CAT',
 'CEBPE',
 'FMO6P',
 'CDRT7',
 'GPHB5',
 'RIMKLA',
 'APMAP',
 'ASB8',
 'CRELD2',
 'YIPF3',
 'SPATS1',
 'ZC3H4',
 'KCNK9',
 'ITFG1',
 'SLC7A8',
 'C1QTNF12',
 'DUPD1',
 'DLGAP1-AS5',
 'KRT17',
 'RNF157-AS1',
 'HOXC-AS2',
 'ACTL6B',
 'SPRR2E',
 'PI4KAP1',
 'CAV1',
 'CTAGE11P',
 'ERGIC2',
 'ANO3',
 'KIAA1841',
 'PANK4',
 'MLC1',
 'RNF181',
 'TMEM100',
 'SNORD160',
 'TTC34',
 'CHD1L',
 'SNRPC',
 'RBL2',
 'SLAMF7',
 'WNT16',
 'FXYD1',
 'LRRC52-AS1',
 'CDK1',
 'GJD3',
 'TEX50',
 'UNC50',
 'TRAPPC9',
 'DUXB',
 'TTYH3',
 'POLE2',
 'LEKR1',
 'DNAJB11',
 'PDE4D',
 'RPE',
 'FABP12',
 'ELK1',
 'RARB-AS1',
 'R3HDM2',
 'ABHD4',
 'GRK6',
 'SPSB1',
 'AKA

In [4]:
df_2

,Unnamed: 0,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,-0.122637,0.025881,0.034217,-0.128082,-0.031285,0.338046,-0.006439,-0.093642,0.189186,...,-0.131727,-0.039829,0.179405,0.283552,0.204513,-0.289724,-0.062972,0.074180,0.111244,-0.467908
1,ACH-000004,0.019756,-0.083640,-0.060118,-0.027417,-0.036116,-0.001056,0.312876,-0.086897,0.204434,...,-0.170329,-0.454263,0.194583,0.098989,0.126948,0.032983,-0.410392,0.113156,0.234388,-0.088306
2,ACH-000005,-0.107208,-0.023211,0.200204,0.116039,-0.172227,-0.071294,0.203270,-0.127806,-0.090981,...,-0.301695,-0.454969,-0.061959,-0.036427,0.016602,-0.201273,-0.178877,-0.055349,-0.002161,-0.186842
3,ACH-000007,-0.031027,-0.137850,0.067704,0.107988,0.007992,0.124945,0.049548,-0.220824,-0.165669,...,-0.303390,-0.507272,-0.025400,0.236659,0.072010,-0.100344,-0.462160,-0.001555,-0.325964,-0.486660
4,ACH-000009,0.008888,-0.146566,0.084471,0.089419,0.065109,0.027841,0.087943,-0.161369,0.041121,...,-0.255466,-0.288739,-0.037132,0.261444,-0.062391,-0.112703,-0.598698,0.095877,-0.026742,-0.320759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,ACH-002834,-0.055721,-0.121228,0.072790,0.021916,-0.221940,0.071135,0.134854,-0.504790,0.058356,...,-0.104959,-0.559438,-0.062461,-0.058814,-0.076221,0.024959,0.015259,-0.306562,-0.148368,-0.579576
1096,ACH-002847,-0.009973,-0.119813,0.044097,0.041358,-0.146565,0.160566,0.134247,-0.242759,-0.085035,...,-0.314751,-0.903238,0.104278,0.101270,0.026373,-0.119911,-0.289412,-0.195097,-0.206400,-0.486525
1097,ACH-002922,-0.025991,-0.007706,-0.038468,0.236576,-0.239690,0.061611,0.128461,-0.498119,0.022097,...,-0.164150,-0.743401,-0.023401,0.174912,-0.230089,-0.079342,-0.347484,-0.085302,-0.095965,-0.346272
1098,ACH-002925,-0.127639,-0.040705,0.134556,-0.047984,-0.116114,0.213144,0.198647,-0.183355,-0.004409,...,-0.275523,-0.898982,-0.050201,0.125480,-0.189114,-0.043555,-0.335270,-0.208063,-0.094741,-0.222404


In [5]:
for index, row in df_2.iterrows():
    model_id = row['Unnamed: 0']
    name = df_3.loc[df_3["ModelID"] == model_id, "CCLEName"].values[0]
    df_2.at[index, 'Unnamed: 0'] = name

In [6]:
df_2

,Unnamed: 0,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,NIHOVCAR3_OVARY,-0.122637,0.025881,0.034217,-0.128082,-0.031285,0.338046,-0.006439,-0.093642,0.189186,...,-0.131727,-0.039829,0.179405,0.283552,0.204513,-0.289724,-0.062972,0.074180,0.111244,-0.467908
1,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,0.019756,-0.083640,-0.060118,-0.027417,-0.036116,-0.001056,0.312876,-0.086897,0.204434,...,-0.170329,-0.454263,0.194583,0.098989,0.126948,0.032983,-0.410392,0.113156,0.234388,-0.088306
2,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.107208,-0.023211,0.200204,0.116039,-0.172227,-0.071294,0.203270,-0.127806,-0.090981,...,-0.301695,-0.454969,-0.061959,-0.036427,0.016602,-0.201273,-0.178877,-0.055349,-0.002161,-0.186842
3,LS513_LARGE_INTESTINE,-0.031027,-0.137850,0.067704,0.107988,0.007992,0.124945,0.049548,-0.220824,-0.165669,...,-0.303390,-0.507272,-0.025400,0.236659,0.072010,-0.100344,-0.462160,-0.001555,-0.325964,-0.486660
4,C2BBE1_LARGE_INTESTINE,0.008888,-0.146566,0.084471,0.089419,0.065109,0.027841,0.087943,-0.161369,0.041121,...,-0.255466,-0.288739,-0.037132,0.261444,-0.062391,-0.112703,-0.598698,0.095877,-0.026742,-0.320759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,PSS131R_Osteosarcoma,-0.055721,-0.121228,0.072790,0.021916,-0.221940,0.071135,0.134854,-0.504790,0.058356,...,-0.104959,-0.559438,-0.062461,-0.058814,-0.076221,0.024959,0.015259,-0.306562,-0.148368,-0.579576
1096,YUHOIN0650_Melanoma_Skin,-0.009973,-0.119813,0.044097,0.041358,-0.146565,0.160566,0.134247,-0.242759,-0.085035,...,-0.314751,-0.903238,0.104278,0.101270,0.026373,-0.119911,-0.289412,-0.195097,-0.206400,-0.486525
1097,SKNMM_BRAIN_BONE,-0.025991,-0.007706,-0.038468,0.236576,-0.239690,0.061611,0.128461,-0.498119,0.022097,...,-0.164150,-0.743401,-0.023401,0.174912,-0.230089,-0.079342,-0.347484,-0.085302,-0.095965,-0.346272
1098,NaN,-0.127639,-0.040705,0.134556,-0.047984,-0.116114,0.213144,0.198647,-0.183355,-0.004409,...,-0.275523,-0.898982,-0.050201,0.125480,-0.189114,-0.043555,-0.335270,-0.208063,-0.094741,-0.222404


In [7]:
df_4 = pd.DataFrame(columns=list(genes))
df_4.insert(0, 'Sample name', df_2['Unnamed: 0'])
df_4

,Sample name,GBAP1,PRODH2,SUMO1P1,CCL27,NOTCH2NLC,AGFG2,TTC23,TMEM167B,FANK1,...,SGSM1,AAAS,RAI1-AS1,STRADA,VSX2,NAA40,KLF14,ZMAT3,TNFRSF13B,RNF157
0,NIHOVCAR3_OVARY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LS513_LARGE_INTESTINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C2BBE1_LARGE_INTESTINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,PSS131R_Osteosarcoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1096,YUHOIN0650_Melanoma_Skin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1097,SKNMM_BRAIN_BONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df_1["Feature ID"][0]

'5637_URINARY_TRACT_amplicon1_ecDNA_1'

In [59]:
df_1.iloc[0, 22]

"['CASC15', 'CDKAL1', 'E2F3', 'HDGFL1', 'NBAT1', 'PRL', 'SOX4']"